In [ ]:
# all necessary includes
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

import os

In [1]:
# additional functions
def TrainClassifier(dataframe):
    model = RandomForestClassifier(
                                    # params
                                    )

In [ ]:
# read input
data = pd.read_csv('.csv')

In [ ]:
# modify input


In [ ]:
# split into validation and training
validation_fraction = 0.3


In [ ]:
# train the model


In [ ]:
# write model to file


In [ ]:
# assess accuracy of the model
